In [1]:
import glob, os
from bs4 import BeautifulSoup as bs
import re
import collections
import json
import io
import fix,parser

In [7]:
def parse(html,publisher):
    global s,s_body
    if publisher=='sag':
        1#article=parser.parse_sag(html)
        s = bs(html, 'lxml').find('head')
        s_body=bs(html, 'lxml').find('body')
        
        try:
            vol_iss_year=s_body.find('div',{"class":"articleJournalNavTitle"}).text.replace('\n','').replace('\t','')
            year=vol_iss_year[-4:]
            iss= vol_iss_year.split('Issue ')[1].split(', ')[0]         
            vol= vol_iss_year.split('Vol ')[1].split(', ')[0]            
        except Exception:
            year=s_body.find('div',{"class":"published-dates"}).text.split('\n')[1][-4:]
            vol='';iss=''
            pass
        
        source= s.find("meta",{"name":"citation_journal_title"})['content']
        title=s.find("meta",{"name":"dc.Title"})['content']
        creators=s.findAll("meta",{"name":"dc.Creator"});names=[c['content'] for c in creators]         
        abstract=s.find("meta",{"name":"dc.Description"}) or ''
        keywords=s.find("meta",{"name":"keywords"}) or ''
        url=s.find("meta",{"name":"dc.Source"})['content']

        
        if abstract is not '':
            abstract= abstract['content']
        if keywords is not '' : 
            keywords= keywords['content']
    
    if publisher=='taf':
        1#article=parser.parse_sag(html)
        s = bs(html, 'lxml').find('head')
        source= s.find("meta",{"name":"citation_journal_title"})['content']
        year=s.find("meta",{"name":"og:description"})['content'][1:5]
        vol=s.find("meta",{"name":"pbContext"})['content'].split('.i')[1].split(';')[0].lstrip('0')
        iss=s.find("meta",{"name":"pbContext"})['content'].split('.v')[1].split('.i')[0].lstrip('0')
        title=s.find("meta",{"name":"dc.Title"})['content']
        creators=s.findAll("meta",{"name":"dc.Creator"});names=[c['content'] for c in creators]
        abstract=s.find("meta",{"name":"dc.Description"}) or ''
        keywords=s.find("meta",{"name":"keywords"}) or ''
        url=s.find("meta",{"name":"pbContext"})['content'].split('article:article:')[1].split(';')[0] or ''
        
        
        names=[' '.join(name.strip().split()) for name in names]
        if url is not '':
            url='http://www.tandfonline.com/doi/'+url
        if keywords is not '':
            keywords=keywords['content']
        if abstract is not '':
            abstract=abstract['content']


    if publisher=='wly':
        #article=parser.parse_wly(html)
        s = bs(html, 'lxml').find('head')
        s_body=bs(html, 'lxml').find('body')
        source=s.find('meta',{'name':'citation_journal_title'})['content']
        year=s.find('meta',{'name':'citation_publication_date'})['content'][:4]
        vol= s.find('meta',{'name':'citation_volume'})['content']
        iss= s.find('meta',{'name':'citation_issue'})['content']
        title=s.find('meta',{'property':'og:title'})['content']
        creators=s.findAll('meta',{'name':'citation_author'}); names=[c['content'] for c in creators]
        abstract=s_body.find('div',{'class':'article-section__content mainAbstract'}) or ''
        keywords_html=s.findAll('meta',{'name':'citation_keywords'}); keywords=', '.join([k['content'] for k in keywords_html])
        url=s.find('meta',{'name':'citation_abstract_html_url'})['content']

        if abstract is not '':
            abstract=abstract.find('p').text

    article={
        'source'   : source,
        'date'     : year,
        'volume'   : vol,
        'issue'    : iss,
        'title'    : title,
        'creator'  : names,
        'abstract' : abstract,        
        'keyword'  : keywords,
        'url'      : url        
    }
    return article

In [8]:
def src2jsonld(publisher, journal_name,base_folder,redif_folder):
    global article,e,f
    folder=base_folder+"/htmls"
    files=os.listdir(folder);print len(files)
    l=[]
    i=0
    jsonld_file=redif_folder+(base_folder+'.redif').replace('/','_')

    try:
        os.remove(jsonld_file)
    except OSError:
        pass
    for f in files:
        i=i+1
        if 'html' in f:
            if i % 10 ==0:
                print i,
            html=open(folder+'/'+f).read()
            article=parse(html,publisher)
            with io.open(jsonld_file, 'a', encoding="utf-8") as f:
                f.write(unicode('Template-Type: ReDIF-Article 1.0\n'))
                f.write(unicode('Title: '+article['title']+'\n'))
                for creator in article['creator']:
                    f.write(unicode('Author-Name: '+creator+'\n'))
                f.write(unicode('Abstract: '+article['abstract']+'\n'))
                f.write(unicode('Year: '+article['date']+'\n'))
                f.write(unicode('Volume: '+article['volume']+'\n'))
                f.write(unicode('Issue: '+article['issue']+'\n'))
                f.write(unicode('Keyword: '+article['keyword']+'\n'))
                f.write(unicode('File-url: '+article['url']+'\n'))
                f.write(unicode('Journal: '+article['source']+'\n'))            
                f.write(unicode('\n'))            
    return (jsonld_file)

In [ ]:
with open('journals.json') as data_file:    
    journals = json.load(data_file)

redif_folder='redif/src/parsed/'

for journal in journals:
    source=journal['jname']
    journal_name=journal['journal']
    data=journal['data']
    for type_pub in data:   
        file_type=type_pub['type']
        publisher=type_pub['publisher']
        if file_type=='src' and publisher in ['taf','wly','sag'][1:3]:
            e=0
            print publisher,source,journal_name,
            base_folder=file_type+'/'+publisher+'/'+source
            src2jsonld(publisher,journal_name,base_folder,redif_folder)
            print 'fixed= ',e

 wly popmgt Production and Operations Management 1538
10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060